In [1]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pprint import pprint

tf.enable_eager_execution()
tf.set_random_seed(777)

/Users/sinseongjin/tf110/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Preparing Dataset

In [2]:
words = ['good', 'bad', 'worse', 'so good']
y_data = [1,0,0,1]
char_set = ['<pad>'] + sorted(list(set(''.join(words))))
idx2char = {idx : char for idx, char in enumerate(char_set)}
char2idx = {char : idx for idx, char in enumerate(char_set)}

print(idx2char)
print(char2idx)

{0: '<pad>', 1: ' ', 2: 'a', 3: 'b', 4: 'd', 5: 'e', 6: 'g', 7: 'o', 8: 'r', 9: 's', 10: 'w'}
{'<pad>': 0, ' ': 1, 'a': 2, 'b': 3, 'd': 4, 'e': 5, 'g': 6, 'o': 7, 'r': 8, 's': 9, 'w': 10}


In [3]:
x_data = list(map(lambda word : [char2idx.get(char) for char in word], words))
x_data_len = list(map(lambda word : len(word), x_data))

print(x_data)
print(x_data_len)

[[6, 7, 7, 4], [3, 2, 4], [10, 7, 8, 9, 5], [9, 7, 1, 6, 7, 7, 4]]
[4, 3, 5, 7]


In [4]:
# padding
max_sequence = 10
x_data = pad_sequences(sequences = x_data, maxlen = max_sequence,
                       padding = 'post', truncating = 'post')

# 데이터 형태 확인
print(x_data)
print(x_data_len)
print(y_data)

[[ 6  7  7  4  0  0  0  0  0  0]
 [ 3  2  4  0  0  0  0  0  0  0]
 [10  7  8  9  5  0  0  0  0  0]
 [ 9  7  1  6  7  7  4  0  0  0]]
[4, 3, 5, 7]
[1, 0, 0, 1]


In [8]:
# input
data = tf.data.Dataset.from_tensor_slices((x_data_len, x_data, y_data))
data = data.shuffle(buffer_size = 4)
data = data.batch(batch_size = 2)

In [9]:
for x_len, x, y in data:
    x_len = x
    x = x
    y = y

In [45]:
# idx2char
n_of_classes = len(idx2char)
hidden_size = 10
batch_size = 2

In [30]:
def gru(units):
    return tf.keras.layers.GRU(units)

def lstm(units):
    return tf.keras.layers.LSTM(units)

In [31]:
class Model(tf.keras.Model):
    def __init__(self, n_of_classes, enc_units, batch_size):
        super(Model, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(n_of_classes, 10)
        self.gru = gru(self.enc_units)
    
    def call(self, x, hidden):
        x = self.embedding(x)
        output = self.gru(x, initial_state = hidden)
        score = keras.layers.Dense(units=n_of_classes)(output)
        
        return score
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [61]:
rnn_model = Model(n_of_classes, hidden_size, batch_size)
hidden = rnn_model.initialize_hidden_state()
score = rnn_model(x, hidden)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
# Loss 함수 정의
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=score)
prediction = tf.argmax(input = score, axis = -1)

In [50]:
for x_len, x, y in data:
    x_len = x_len
    x = x
    y = y

In [72]:
import time

EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = rnn_model.initialize_hidden_state()
    total_loss = 0
    
    for x_len, x, y in data:
                
        with tf.GradientTape() as tape:
            hidden = rnn_model.initialize_hidden_state()
            score = rnn_model(x, hidden)
            loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=score)
        grad = tape.gradient(loss, rnn_model.variables)
        optimizer.apply_gradients(zip(grad, rnn_model.variables),
                                 global_step=tf.train.get_or_create_global_step())

        print(epoch)

        
    total_loss += loss
    
    

0


In [70]:
total_loss

<tf.Tensor: id=31368, shape=(), dtype=float32, numpy=2.410603>